In [31]:
import matplotlib
import csv
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import style, colors
from matplotlib import gridspec

matplotlib.style.use('ggplot')
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (8, 6)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

DEBUG:matplotlib.backends:backend module://ipykernel.pylab.backend_inline version unknown


### snv and indel vcf files

In [36]:
f = '/projects/trans_scratch/validations/workspace/szong/Cervical/variant_bwamem/124_patients_new/124_patients_bam_vcf_20190115.csv'
df = pd.read_csv(f, usecols=['patient','status', 'strelka_snv_vcf', 'strelka_indel_vcf'])
df = df[df.status == 'Malignant']
df.drop('status', axis=1, inplace=True)
df.shape
df.head(2)

(124, 3)

,patient,strelka_indel_vcf,strelka_snv_vcf
0,HTMCP-03-06-02001,/projects/somatic/NCI_SAIC_HIV_Cervical/HTMCP-...,/projects/somatic/NCI_SAIC_HIV_Cervical/HTMCP-...
1,HTMCP-03-06-02002,/projects/somatic/NCI_SAIC_HIV_Cervical/HTMCP-...,/projects/somatic/NCI_SAIC_HIV_Cervical/HTMCP-...


In [37]:
df['strelka_indel_vcf'][0]

'/projects/somatic/NCI_SAIC_HIV_Cervical/HTMCP-03-06-02001/hg19a/GSH/A37234_A37159/strelka/59844/bwa/results/passed.somatic.indels.eff.dbSNP_v149.cosmic_v82.clinvar_20170801.annotations.classic.vcf'

In [38]:
f = '/projects/trans_scratch/validations/workspace/szong/Cervical/mutsig2cv/118_patients.txt'
df1 = pd.read_csv(f,header=None)
df1.columns = ['patient']
df1.head()
df1.shape
patients = df1['patient']
patients[:2]

,patient
0,HTMCP-03-06-02001
1,HTMCP-03-06-02002
2,HTMCP-03-06-02003
3,HTMCP-03-06-02006
4,HTMCP-03-06-02008


(118, 1)

0    HTMCP-03-06-02001
1    HTMCP-03-06-02002
Name: patient, dtype: object

In [39]:
df = df[df.patient.isin(patients)]
df.columns = ['patient', 'indels', 'snvs']
df.shape

(118, 3)

In [40]:
df.head(2)

,patient,indels,snvs
0,HTMCP-03-06-02001,/projects/somatic/NCI_SAIC_HIV_Cervical/HTMCP-...,/projects/somatic/NCI_SAIC_HIV_Cervical/HTMCP-...
1,HTMCP-03-06-02002,/projects/somatic/NCI_SAIC_HIV_Cervical/HTMCP-...,/projects/somatic/NCI_SAIC_HIV_Cervical/HTMCP-...


### template vcf2maf scripts

In [41]:
from jinja2 import Environment, FileSystemLoader
import logging
import colorlog

logger = colorlog.getLogger()
logger.setLevel(logging.DEBUG)
handler = logging.StreamHandler()
handler.setLevel(logging.DEBUG)
handler.setFormatter(
    colorlog.ColoredFormatter('%(log_color)s%(levelname)s:%(name)s:%(message)s'))
logger.addHandler(handler)


In [42]:
template_dir = '/projects/trans_scratch/validations/workspace/szong/Cervical/mutsig2cv/'
template_file_name = 'vcf2maf_template_discovery.txt'
out_maf_path = '/projects/trans_scratch/validations/workspace/szong/Cervical/mutsig2cv/118_patients/'

In [43]:
def populate_template(patient, input_vcf, output_maf, 
                      template_dir, template_file_name, templated_file_name):
    jinja2_env = Environment(loader=FileSystemLoader([template_dir]),
                             trim_blocks=True)
    template = jinja2_env.get_template(template_file_name)
    with open(templated_file_name, 'w') as opf:
        content = template.render(patient = patient,
                                  input_vcf = input_vcf,
                                  output_maf = output_maf)
        opf.write(content)
        logger.info('templated {0}'.format(templated_file_name))
    return template_file_name


In [45]:
for vcf_type in ['snvs', 'indels']:  
    print(f'processing {vcf_type}')
    for ix,row in df.iterrows():
        patient = row.patient
        input_vcf = row[vcf_type]
        print(input_vcf)
        output_maf = '{0}mafs/{1}.{2}.maf'.format(out_maf_path, patient, vcf_type)
        templated_file_name = '/projects/trans_scratch/validations/workspace/szong/Cervical/mutsig2cv/118_patients/vcf2maf_scripts/{0}.{1}.sh'.format(patient, vcf_type)
#         print(patient, input_vcf, output_maf, templated_file_name)
        populate_template(patient, input_vcf, output_maf, 
                          template_dir, template_file_name, 
                          templated_file_name)

INFO:root:templated /projects/trans_scratch/validations/workspace/szong/Cervical/mutsig2cv/118_patients/vcf2maf_scripts/HTMCP-03-06-02001.snvs.sh
INFO:root:templated /projects/trans_scratch/validations/workspace/szong/Cervical/mutsig2cv/118_patients/vcf2maf_scripts/HTMCP-03-06-02001.snvs.sh


processing snvs
/projects/somatic/NCI_SAIC_HIV_Cervical/HTMCP-03-06-02001/hg19a/GSH/A37234_A37159/strelka/59844/bwa/results/passed.somatic.snvs.eff.dbSNP_v149.cosmic_v82.clinvar_20170801.annotations.classic.vcf


'vcf2maf_template_discovery.txt'

INFO:root:templated /projects/trans_scratch/validations/workspace/szong/Cervical/mutsig2cv/118_patients/vcf2maf_scripts/HTMCP-03-06-02002.snvs.sh
INFO:root:templated /projects/trans_scratch/validations/workspace/szong/Cervical/mutsig2cv/118_patients/vcf2maf_scripts/HTMCP-03-06-02002.snvs.sh


/projects/somatic/NCI_SAIC_HIV_Cervical/HTMCP-03-06-02002/hg19a/GSH/A37235_A37160/strelka/59848/bwa/results/passed.somatic.snvs.eff.dbSNP_v149.cosmic_v82.clinvar_20170801.annotations.classic.vcf


...

/projects/somatic/NCI_SAIC_HIV_Cervical/HTMCP-03-06-02442/hg19a/GSH/A78437_A76750/strelka/60141/bwa/results/passed.somatic.indels.eff.dbSNP_v149.cosmic_v82.clinvar_20170801.annotations.classic.vcf


'vcf2maf_template_discovery.txt'

INFO:root:templated /projects/trans_scratch/validations/workspace/szong/Cervical/mutsig2cv/118_patients/vcf2maf_scripts/HTMCP-03-06-02447.indels.sh
INFO:root:templated /projects/trans_scratch/validations/workspace/szong/Cervical/mutsig2cv/118_patients/vcf2maf_scripts/HTMCP-03-06-02447.indels.sh


/projects/somatic/NCI_SAIC_HIV_Cervical/HTMCP-03-06-02447/hg19a/GSH/A78440_A76753/strelka/60142/bwa/results/passed.somatic.indels.eff.dbSNP_v149.cosmic_v82.clinvar_20170801.annotations.classic.vcf


'vcf2maf_template_discovery.txt'

### convert vcfs to mafs

ssh to gphost06, and bash all the .sh scripts

for f in *snvs.sh;do bash $f;done

for f in *indels.sh;do bash $f;done

### run mutsig2cv